In [ ]:
from confluent_kafka import Producer
import json
import random
import time

producer = Producer({'bootstrap.servers': 'localhost:9092'})

actions = ["login", "click", "purchase", "logout"]

def generate_event():
    return {
        "user_id": random.randint(1, 10),
        "action": random.choice(actions),
        "timestamp": time.strftime('%Y-%m-%dT%H:%M:%S')
    }

while True:
    event = generate_event()
    producer.produce('example_topic', json.dumps(event))
    producer.flush()
    print(f"Produced: {event}")
    time.sleep(1)


In [ ]:
from confluent_kafka import Consumer
import json
from collections import Counter

consumer = Consumer({
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'example_group',
    'auto.offset.reset': 'earliest'
})

consumer.subscribe(['example_topic'])

action_counts = Counter()

try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            print(f"Consumer error: {msg.error()}")
            continue

        event = json.loads(msg.value().decode('utf-8'))
        user_id = event['user_id']
        action = event['action']
        action_counts[(user_id, action)] += 1

        print(f"Received: {event}")

        # Анализ: топ пользователей по количеству действий
        top_users = Counter({k[0]: v for k, v in action_counts.items() if k[1] in ["click", "purchase"]})
        print(f"Top users by actions: {top_users.most_common(3)}")

finally:
    consumer.close()
